In [ ]:
import plotly.express as px
df3 = data_set['Pawpularity'] 
fig = px.histogram(df3, x="Pawpularity")
fig.show()

In [ ]:
import plotly.express as px
fig = px.box(df3, y="Pawpularity")
fig.show()

In [ ]:
number=data_set.loc[data_set.Pawpularity > 80 , 'Pawpularity'].count()
(number/data_set.shape[0])*100
#no need for hashing to select 0.2 of data since we won't be fetching new datasets
#we do it again with sklearn

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data_set, test_size=0.2, random_state=42)
##data is representative after a random split? Think not.
# homogeneous data = strata; ensure that the test set is representative of the various categories of incomes in the whole dataset.
#it kinda works somehow for pawpularity but we need ensurance for all categories
data_set["pawtegory"] = pd.cut(data_set["Pawpularity"],
bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90,  np.inf],
labels=[1, 2, 3, 4, 5,6,7,8,9,10])
data_set["pawtegory"].hist()
from sklearn.model_selection import StratifiedShuffleSplit

split= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data_set, data_set["pawtegory"]):
    strat_train_set = data_set.loc[train_index]
    strat_test_set = data_set.loc[test_index]

strat_test_set["pawtegory"].value_counts() / len(strat_test_set)
# lets compate it to our raw data
data_set["pawtegory"].value_counts() / len(data_set)
#strat error
((strat_test_set["pawtegory"].value_counts() / len(strat_test_set)) -  (data_set["pawtegory"].value_counts() / len(data_set)) )  *100

# deleting pawtegory

for set_ in (strat_train_set, strat_test_set):
    set_.drop("pawtegory", axis=1, inplace=True)
    # Discover and Visualize the Data to Gain Insights
cuteness= strat_train_set.copy()
#couldnt visualize it with scatter
# linear correlation
corr_matrix = cuteness.corr()
corr_matrix["Pawpularity"].sort_values(ascending=False)
#it proves that these data dont really have a huge influence on the pawpularity, 
#we should consider dimension reduction and merging features 
#correlation between attributes
#'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory','Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur','Pawpularity'
#copying into already named folders
import shutil
import os
for name in strat_test_set["Id"]:
    file = os.path.join(r"C:\Users\Yas\Downloads\petfinder-pawpularity-score\train", name + ".jpg")
    shutil.copy2( file, r"C:\Users\Yas\Desktop\test")


for name in strat_train_set["Id"]:
    file = os.path.join(r"C:\Users\Yas\Downloads\petfinder-pawpularity-score\train", name + ".jpg")
    shutil.copy2( file, r"C:\Users\Yas\Desktop\train")
#separate the predictors and the labels


cuteness= strat_train_set.drop(["Pawpularity", "Id"], axis=1)

cuteness_label = strat_train_set["Pawpularity"].copy()

cuteness.info()

    from sklearn.tree import DecisionTreeRegressor
    tree_reg = DecisionTreeRegressor()
    tree_reg.fit(cuteness, cuteness_label)

from sklearn.metrics import mean_squared_error
cuteness_predictions = tree_reg.predict(cuteness)
tree_mse = mean_squared_error(cuteness_label, cuteness_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse


from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, cuteness, cuteness_label, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)


def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)